In [63]:
from configparser import ConfigParser
import psycopg2
import pandas as pd
import os
import glob
import numpy as np
from sql_queries import *

In [64]:
# The below function creates an instance of the ConfigParsr class and makes use of its methods to read the config file

In [65]:
def get_connection_byconfig(filepath,section):
    if (len(filepath)>0 and len(section)>0):
# creates instance of the Config Parser class
        parser=ConfigParser()
# reads the configuration file
        parser.read(filepath)
        diction={}
# if the configuration file has the passed section name
        if (parser.has_section(section)):
# this creates each line in the config file has a separate list. Example ('host','localhost')
            config_params=parser.items(section)
# convert list to python dictionary object
# create empty dictionary
          
            for c in config_params:
                key=c[0]
                value=c[1]
                diction[key]=value
        return diction

     

In [66]:
d=get_connection_byconfig('Config_ini','postgresql')

In [67]:
conn = psycopg2.connect(host=d['host'], user=d['user'], dbname="sparkifydb",password=d['password'],port=d['port'])
cur = conn.cursor()

In [68]:
def get_files(filepath):
    all_files = []
# os.walk method generates filenames in a directory tree
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    return all_files


def insert_from_dataframe(df, insert_query):
    for i, row in df.iterrows():
        cur.execute(insert_query, list(row))
    conn.commit()
 

In [69]:
song_files = get_files('/home/dale/Data Engineer Project/song-data/song_data/')

In [70]:
len(song_files)

71

In [71]:
df=pd.DataFrame()
for song_file in song_files:
    df = df.append(pd.read_json(song_file, lines=True), ignore_index=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARD7TVE1187B99BFB1,NaN,NaN,California - LA,Casual,SOMZWCG12A8C13C480,I Didn't Mean To,218.93179,0
1,1,ARMJAGH1187FB546F3,35.14968,-90.04892,"Memphis, TN",The Box Tops,SOCIWDW12A8C13D406,Soul Deep,148.03546,1969
2,1,ARKRRTF1187B9984DA,NaN,NaN,,Sonora Santanera,SOXVLOJ12AB0189215,Amor De Cabaret,177.47546,0
3,1,AR7G5I41187FB4CE6C,NaN,NaN,"London, England",Adam Ant,SONHOTT12A8C13493C,Something Girls,233.40363,1982
4,1,ARXR32B1187FB57099,NaN,NaN,,Gob,SOFSOCN12A8C143F5D,Face the Ashes,209.60608,2007


In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   num_songs         71 non-null     int64  
 1   artist_id         71 non-null     object 
 2   artist_latitude   31 non-null     float64
 3   artist_longitude  31 non-null     float64
 4   artist_location   71 non-null     object 
 5   artist_name       71 non-null     object 
 6   song_id           71 non-null     object 
 7   title             71 non-null     object 
 8   duration          71 non-null     float64
 9   year              71 non-null     int64  
dtypes: float64(3), int64(2), object(5)
memory usage: 5.7+ KB


In [73]:
artist_data=df[['artist_id','artist_name','artist_latitude','artist_longitude','artist_location']]

In [74]:
artist_data=artist_data.drop_duplicates()

In [75]:
artist_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 70
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         69 non-null     object 
 1   artist_name       69 non-null     object 
 2   artist_latitude   31 non-null     float64
 3   artist_longitude  31 non-null     float64
 4   artist_location   69 non-null     object 
dtypes: float64(2), object(3)
memory usage: 3.2+ KB


In [76]:
artist_data = artist_data.replace(np.nan, None, regex=True)

In [77]:
artist_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69 entries, 0 to 70
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   artist_id         69 non-null     object 
 1   artist_name       69 non-null     object 
 2   artist_latitude   68 non-null     float64
 3   artist_longitude  68 non-null     float64
 4   artist_location   69 non-null     object 
dtypes: float64(2), object(3)
memory usage: 3.2+ KB


In [78]:
insert_from_dataframe(artist_data,artists_insert_table)

In [79]:
song_data = df[['song_id','title', 'artist_id', 'duration', 'year']]
song_data = song_data.drop_duplicates()
song_data = song_data.replace(np.nan, None, regex=True)
song_data.head()

,song_id,title,artist_id,duration,year
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,218.93179,0
1,SOCIWDW12A8C13D406,Soul Deep,ARMJAGH1187FB546F3,148.03546,1969
2,SOXVLOJ12AB0189215,Amor De Cabaret,ARKRRTF1187B9984DA,177.47546,0
3,SONHOTT12A8C13493C,Something Girls,AR7G5I41187FB4CE6C,233.40363,1982
4,SOFSOCN12A8C143F5D,Face the Ashes,ARXR32B1187FB57099,209.60608,2007


In [80]:
insert_from_dataframe(song_data,songs_insert_table)

In [81]:
log_files = get_files('/home/dale/Data Engineer Project/log-data/')

In [82]:
df=pd.DataFrame()
for log_files in log_files:
    df = df.append(pd.read_json(log_files, lines=True), ignore_index=True)
df

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,None,Logged In,Walter,M,0,Frye,NaN,free,"San Francisco-Oakland-Hayward, CA",GET,Home,1.540919e+12,38,None,200,1541105830796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",39
1,None,Logged In,Kaylee,F,0,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Home,1.540345e+12,139,None,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
3,None,Logged In,Kaylee,F,2,Summers,NaN,free,"Phoenix-Mesa-Scottsdale, AZ",GET,Upgrade,1.540345e+12,139,None,200,1541106132796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8051,Timbiriche,Logged In,Rylan,M,58,George,202.60526,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Besos De Ceniza,200,1543603476796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8052,A Perfect Circle,Logged In,Rylan,M,59,George,206.05342,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,Rose,200,1543603678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8053,Anberlin,Logged In,Rylan,M,60,George,348.68200,paid,"Birmingham-Hoover, AL",PUT,NextSong,1.541020e+12,1076,The Haunting,200,1543603884796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16
8054,None,Logged In,Rylan,M,61,George,NaN,paid,"Birmingham-Hoover, AL",GET,Downgrade,1.541020e+12,1076,None,200,1543603993796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",16


In [83]:

df = df[df['page'] == 'NextSong']
df = df.replace(np.nan, None, regex=True)
df.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
2,Des'ree,Logged In,Kaylee,F,1,Summers,246.30812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,You Gotta Be,200,1541106106796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
4,Mr Oizo,Logged In,Kaylee,F,3,Summers,144.03873,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Flat 55,200,1541106352796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
5,Tamba Trio,Logged In,Kaylee,F,4,Summers,177.18812,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Quem Quiser Encontrar O Amor,200,1541106496796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
6,The Mars Volta,Logged In,Kaylee,F,5,Summers,380.42077,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Eriatarka,200,1541106673796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8
7,Infected Mushroom,Logged In,Kaylee,F,6,Summers,440.26730,free,"Phoenix-Mesa-Scottsdale, AZ",PUT,NextSong,1.540345e+12,139,Becoming Insane,200,1541107053796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",8


In [84]:

tf = pd.DataFrame({
    'start_time': pd.to_datetime(df['ts'],unit='ms')
})


tf['hour'] = tf['start_time'].dt.hour
tf['day'] = tf['start_time'].dt.day
tf['week'] = tf['start_time'].dt.week
tf['month'] = tf['start_time'].dt.month
tf['year'] = tf['start_time'].dt.year
tf['weekday'] = tf['start_time'].dt.weekday

tf = tf.drop_duplicates()

tf.head()



,start_time,hour,day,week,month,year,weekday
2,2018-11-01 21:01:46.796,21,1,44,11,2018,3
4,2018-11-01 21:05:52.796,21,1,44,11,2018,3
5,2018-11-01 21:08:16.796,21,1,44,11,2018,3
6,2018-11-01 21:11:13.796,21,1,44,11,2018,3
7,2018-11-01 21:17:33.796,21,1,44,11,2018,3


In [85]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6813 entries, 2 to 8055
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   start_time  6813 non-null   datetime64[ns]
 1   hour        6813 non-null   int64         
 2   day         6813 non-null   int64         
 3   week        6813 non-null   int64         
 4   month       6813 non-null   int64         
 5   year        6813 non-null   int64         
 6   weekday     6813 non-null   int64         
dtypes: datetime64[ns](1), int64(6)
memory usage: 425.8 KB


In [86]:
insert_from_dataframe(tf, time_insert_table)

In [87]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df = user_df.drop_duplicates()
user_df = user_df[user_df['userId'] != '']

user_df.head()

,userId,firstName,lastName,gender,level
2,8,Kaylee,Summers,F,free
10,10,Sylvie,Cruz,F,free
12,26,Ryan,Smith,M,free
14,101,Jayden,Fox,M,free
17,83,Stefany,White,F,free


In [88]:
insert_from_dataframe(user_df, users_insert_table)

In [89]:

for index, row in df.iterrows():
    print(row.song)
    cur.execute(song_select,(row.song,row.artist,row.length))
    results = cur.fetchone()
    print(results)
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    

You Gotta Be
None
Flat 55
None
Quem Quiser Encontrar O Amor
None
Eriatarka
None
Becoming Insane
None
Congratulations
None
Once again
None
Pump It
None
Nobody Puts Baby In The Corner
None
Mango Pickle Down River (With The Wilcannia Mob)
None
Eye Of The Tiger
None
Am I High (Feat. Malice)
None
A Lack Of Color (Album Version)
None
I Have A Wish
None
Monster (Album Version)
None
Uneasy Hearts Weigh The Most
None
Falta Te Dizer
None
Family Business
None
Lucky (Album Version)
None
Journey To The Past  (LP Version)
None
Everything I Try to Do_ Nothing Seems to Turn Out Right
None
Impossible
None
Secrets
None
Wild World
None
World Keeps Turning
None
Greed
None
Swing Javanaise
None
Ed Ero Contentissimo
None
Ship Inside A Bottle
None
The Cut
None
Ride A White Swan
None
Mirando a Las Muchachas
None
Clouds (Of Color Bright Album Version)
None
Riberonzinha
None
Big Blue Note
None
Naked
None
In All This Revelation
None
Bottom of a Bottle (Explicit Album Version)
None
Wake Up Alone
None
Sisterfucker 

None
Hold On
None
Do You Remember
None
In Shadows And Dust
None
Everybody's Changing
None
Cosmic Love
None
Taste In Men
None
Sexy Bitch
None
Anti-Western
None
Spell
None
God Is On The Radio
None
Bleed It Out [Live At Milton Keynes]
None
You're The One
None
Bullet To Binary (Pt. Two)
None
All The Things That Go To Make Heaven And Earth
None
Home
None
SueÃÂ±os
None
MuÃÂ©rdete la lengua
None
The Other Side (album version)
None
Shape Shifter
None
Emotion
None
Danza Rota
None
Lovefool
None
I Wanna Be The Only One
None
Revelry
None
Me and My Friend
None
Cornman
None
Una Confusion
None
Invalid
None
The Lighthouse's Tale
None
One Love
None
Strangers In The Wind
None
Existentialism On Prom Night (Album Version)
None
Echame A Mi La Culpa
None
Blind (Full Album Version)
None
Amerika
None
Come Back Darling
None
Fate Of Norns
None
Silent Lucidity (2002 Digital Remaster)
None
Reloaded
None
Ben
None
My Moon My Man
None
Heartbeats
None
Handlebars (UK Radio Edit)
None
Heartbreaker
None
You're The One

None
Build High
None
Atonement
None
A Beggar On A Beach Of Gold
None
Music Matters (Mark Knight Dub)
None
Be Somebody
None
Carnal
None
Impossible
None
Bare
None
Smile
None
Sehr kosmisch
None
Keep Holding On
None
When The Children Cry
None
You And Me (Wedding Version)
None
Stranger Aeons
None
The Climb
None
Indestructible
None
Lanky (Part One)
None
Racing East
None
Angel
None
Sugar Ray (LP Version)
None
Guaranteed
None
No Cars Go
None
Verde Que Te Quero Rosa
None
Good Man
None
Tomorrow
None
You're The One
None
Halo
None
Take Me Out
None
Itchy Feet
None
Aerodynamic (Daft Punk Remix)
None
Narc
None
On A Ragga Tip
None
It Is You - Live
None
While You're Still Young
None
Hotel
None
shimmer
None
Master Of Puppets
None
Nothin' On You [feat. Bruno Mars] (Album Version)
None
Why Georgia
None
Ghost Town
None
Brooklyn Zoo [Explicit Version]
None
Bubble Toes
None
You
None
Nobody Puts Baby In The Corner
None
Not Fair (Clean Radio Edit)
None
Section Ten
None
Macarena
None
ReprÃÂ©sente
None
Second C

None
Estoy AquÃÂ­
None
Yeah!
None
Hanuman
None
I Wanna Be Sedated (Remastered Album Version )
None
Miserable At Best (Album)
None
Music Box
None
Visions
None
Calyx Virago
None
Drop The Mirror (EP Version)
None
Shook Ones pt. 2
None
Pretendo Hablarte
None
Bring Me To Life
None
Clear Blue Water
None
You're The One
None
Lord Xenu
None
Take Me To The River
None
Flat Beat
None
Never Too Far
None
Day 'N' Nite
None
El Hombre
None
Geek In The Pink [Phil Tan Remix]
None
You're The One
None
I Second That Emotion
None
Say The Same
None
Why Aye Man
None
We're No Here
None
Punx Unite
None
Roll On (Album Version)
None
I Need A Dollar
None
Helpless
None
Sunshower (Great Expectations Soundtrack)
None
My Name Is Jonas
None
Valley Of The Queens
None
Shining Down [feat. Matthew Santos] (Amended Album Version)
None
Runnin' Down A Dream
None
When You Were Young
None
I'm Her Slave (Album)
None
Let's Make Love And Listen To Death From Above [Dan Carey Mix] (remastered album version)
None
History
None
How It

None
Fire Coming Out Of The Monkey's Head
None
Ready_ Steady_ Go (Album Version)
None
Better Off Dead
None
Boys Will Be Boys
None
Hey_ Soul Sister
None
My Perfect Cousin
None
God Is On The Radio
None
Between The Bars
None
Seven Nation Army
None
Signals Over The Air
None
Susanne
None
Sexy Eyes
None
You're The One
None
La Otra Princesa
None
McFearless
None
Rayando el sol
None
Genom tunna tyger
None
Cards & Quarters
None
Bulletproof
None
Seerosenteich
None
Ripped To Shreds (feat. Vinnie Paz_ Celph Titled & Demoz)
None
Push The Feeling On
None
Say Back Something
None
Careful (Album Version)
None
Guys Like Me
None
Petit Papa Noel (Album Version)
None
Good Directions
None
Shiny & New
None
Kun Puut Tekee SeittiÃÂ¤
None
Mama Tierra
None
M1 A1
None
Slang Blade
None
Aberinkula
None
Time To Pretend
None
Don't Stop At The Top
None
Not An Addict
None
Human
None
Suspicious Minds
None
Good Things
None
Moon And Moon
None
If I Had You
None
Sayonara-Nostalgia
None
No No No
None
Passenger
None
You And M

...

None
Portobello Road
None
Surrendering (Album Version)
None
Anytime You Need A Friend
None
Mercy:The Laundromat
None
Girls And Boys In Love
None
The Beautiful People
None
Time Bomb
None
Coleccionista De Canciones
None
Lose Control
None
Paa vauhtii
None
221
None
The Perfect Space
None
Mother Mother
None
Canada
None
(Nice Dream)
None
Magic
None
Make Me Better
None
Minimum
None
Mon Ange
None
Mr. Jones
None
Money To Burn
None
Prayers
None
Die Kunst der Fuge_ BWV 1080 (2007 Digital Remaster): Contrapunctus XVII - Inversus
None
Revolution
None
I Might Be Wrong
None
Proper Education
None
Christeen
None
TRAS
None
Party All Over Your Body
None
Possessed
None
Can't Let Go
None
Superbeast
None
Picture
None
Caca E Cacador
None
Space Travel
None
How Many Times_ How Many Lies
None
One Time
None
Always Like This
None
Nothin' On You [feat. Bruno Mars] (Album Version)
None
You Can Do Better Than Me (Album Version)
None
Me_ Myself And I
None
Cover Me
None
Like A Stone
None
Publish My Love (Album Version

None
Human Patterns
None
Dog Days Are Over (Radio Edit)
None
Dry Spell (LP Version)
None
Hardest Of Hearts
None
Fix You
None
In My Place
None
Two Is Better Than One
None
Summerland
None
Always (7'' Mix)
None
This Is The Life
None
Saving Us (Album Version)
None
Jumento Celestino
None
What Do I Know
None
Ela E Carioca
None
La Huella De Tu Mirada
None
What Can I Do?
None
Ready Or Not
None
Behind The Moon
None
Behind Blue Eyes
None
Hardest Of Hearts
None
Black Hole
None
Baby
None
Catch You Baby (Steve Pitron & Max Sanna Radio Edit)
None
Canada
None
Walk Away (Single/LP Version)
None
Undo
None
Love Dog
None
Kiss (LP Version)
None
The Only Exception (Album Version)
None
Le Vin De L'Assassin
None
Dejame Bailar
None
Pursuit Of Happiness (nightmare)
None
Madrugada
None
Living In A Lie
None
Night Divine (On A Search In America Album Version)
None
Never Let You Go
None
Diduntdidunt (featuring Saigon) (Amended Version)
None
Look No Further
None
Heartbeats
None
With You
None
Song For Guy
None
Rabbi

None
No Fit State
None
Missing (Live in Europe)
None
Deep Inside (Harry Choo Choo Romero's Bambossa Remix)
None
Burn Out
None
Politik
None
Human After All
None
Savior
None
Love Will Keep Us Together
None
Purple Weed
None
The Remedy (I Won't Worry) (New EQ'd LP Version)
None
Run Run
None
Ink My Whole Body
None
Wayli Aah
None
Closer To Love
None
Victoria
None
Sex And Violence
None
Livin' On A Prayer
None
Epic
None
Eye Of The Beholder
None
Reichswehr In Blood
None
Victory
None
OMG
None
Why Worry
None
Wait For Me
None
To Be With You
None
Blind (Frankie Knuckles Remix)
None
Here Without You
None
Stuck On The Moon
None
What It Ain't
None
Dear John
None
Me Voy Pal Party
None
Available [feat. Akon] (Album Version)
None
Lower Your Eyelids To Die With The Sun
None
Sound Of Everything Feat. Alice Russell
None
Dustbowl
None
Already Gone
None
Perfection Or Vanity-Outro
None
Birthday Sex
None
Emergency (Album Version)
None
Fireworks
None
Jazz Thieves
None
Pa' Que Retozen (Amended Version)
None
The L

None
Ain't Misbehavin
None
Hello
None
Love Story
None
Heroes
None
Velandote
None
Bossa Nova Cha Cha
None
Get Your Hands Off My Man (Nush Chocolate Factory Mix)
None
Two Angels
None
Help I'm Alive
None
Medicine Ball
None
All Of Us
None
Keep You Much Longer
None
Wide Eyes
None
Back In The Day
None
Where Happiness Lives
None
Something
None
Lady (Hear Me Tonight)
None
I CAN'T GET STARTED
None
Inertia
None
Rainy Night In Tokyo (Remastered LP Version)
None
The Rain  [Supa Dupa Fly]  (LP Version)
None
Que soy yo para ti (Radio edit)
None
Elevator [Feat. Timbaland] (Album Version)
None
Rio
None
Peep Show
None
Sail Into the Sun
None
Particle Man (LP Version)
None
Cheryl Tweedy
None
Sweetheart
None
Midnight Groovin' (7" Version) (2006 Digital Remaster)
None
Battle Song
None
Addams Groove
None
Eutanasia
None
Fix You
None
Fix You (Live)
None
Juicebox
None
Dance_ Dance
None
At The Bottom Of Everything
None
Irreplaceable
None
Shining On (feat. Scott Grella) (Grella's Techno Remix)
None
Ghost Of The 

None
To Be With You
None
Poison Oak (Album Version)
None
Grand Hotel
None
Planet Telex
None
SinceritÃÂ© Et Jalousie
None
New Born
None
5_ 6_ 7_ 8
None
The Heart Of Life
None
Revelry
None
Draw The Line
None
The Ugly One (AKA Lee Van Cleef)
None
Invalid
None
Cathar Rhythm
None
The Queen of Nothing
None
Good Time
None
Figures
None
Bone Broke
None
Are You The Walrus?
None
Amityville
None
I'm Not Calling You A Liar
None
Signos
None
You Belong With Me
None
Rearviewmirror
None
Me So Horny
None
Loco
None
Midnight Groovin' (7" Version) (2006 Digital Remaster)
None
Jim Henson's Dead
None
Shell Shock
None
Women Lose Weight (Feat: Slick Rick)
None
Won't Go Home Without You
None
Hey_ Soul Sister
None
I'm In Miami Bitch
None
Sexy Bitch
None
The Cure & The Cause (Dennis Ferrer Remix)
None
Staring At Me
None
The Second You Sleep
None
Wide Eyes
None
La Resistance (Medley) (LP Version)
None
Restless
None
Noc...
None
Like You
None
God Put A Smile Upon Your Face
None
Playboy
None
Every Time You Go
None
N

In [90]:

for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (pd.to_datetime(row.ts,unit='ms'),row.userId,row.level,songid,artistid,row.sessionId,row.location,row.userAgent)
#     songplay_data=df_songplay.values.tolist()[0]
#     songplay_data
#     songplay_data = (df['ts'],df['userId'],df['level'],)
    cur.execute(songplay_insert_table, songplay_data)
    conn.commit()          
               

In [91]:
conn.close()